Here I will start to rebuild the DDPM paper from scratch, the whole procedure of DDPM procedure can be thought as a process of demolishing a big building into bricks and rebuild the building by using those bricks, we first add noise into the model, then the model will learn how to rebuild the image back into the original one, the DDPM is actually not a typical "diffusion model" but a VAE model.

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [ ]:
class SinusodialPostionalEncoding(nn.Module):
    def __init__(self, dim, theta = 10000):
        super(SinusodialPostionalEncoding, self).__init__()
        self.dim = dim
        self.theta = theta
    
    def positional_encoding(self, x, position):
        """
        An different implementation from original paper 
        'Attention is all you need', this is for computation efficiency
        """
        device = x.device
        position = x[:, None]  # Adding an extra dimension for broadcasting
        div_term = torch.exp(torch.arange(0, self.dim, 2, device=device) * -(math.log(self.theta) / self.dim))

        # Apply sinusoidal functions alternately across dimensions
        pe = torch.zeros(len(x), self.dim, device=device)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        return pe
